# Experiments

## Overview
This notebook contains preliminary experiments comparing different AI agent configurations. Each scenario will use identical study material input.

## Scenarios
There will be 4 scenarios:
1. **Single agent 0-shot**: One agent with no examples provided
2. **Single agent 1-shot**: One agent with one example provided
3. **Multi-agent 0-shot**: Two agents (question generator and evaluator) with no examples, using manual agent orchestration
4. **Multi-agent 1-shot**: Two agents with one example, using manual agent orchestration

## Methodology
- Each scenario will have same study material input
- Each scenario will be run once as this is a preliminary study
- The multi-agent scenarios will utilize the crewAI framework
- Results will be compared qualitatively rather than statistically


In [1]:
%pip install google-genai


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Base Setup

In [2]:
import os
from google import genai
from google.genai import types
import json

In [3]:
base_model = "gemini-2.0-flash"
api_key = os.environ.get("GEMINI_API_KEY")
client = genai.Client(
  api_key=api_key,
)

# setup study material files
files = [
  client.files.upload(file='section-loop.pdf')
]

## Scenario 1 & 2 Setup

In [56]:
question_generator_prompt = ""
with open('question-generator-prompt.txt', 'r') as file:
    question_generator_prompt = file.read()

def setup_question_generator_config(system_prompt):
    return types.GenerateContentConfig(
        response_mime_type="application/json",
        system_instruction=[
            types.Part.from_text(text=system_prompt),
        ],
    )

question_generator_config_0_shot = setup_question_generator_config(question_generator_prompt)

## Scenario 1

In [57]:
contents_scenario_1 = [
    types.Content(
        role="user",
        parts=[
            types.Part.from_uri(
                file_uri=files[0].uri,
                mime_type=files[0].mime_type,
            ),
            types.Part.from_text(text="Generate exactly 5 MCQs covering the different loop concepts and examples presented **in the provided file**. Use the JSON format specified in your instructions."),
        ]
    )
]

In [58]:
result = client.models.generate_content(
  model=base_model,
  contents=contents_scenario_1,
  config=question_generator_config_0_shot,
)

In [59]:
print(f"{result.text}")

[
  {
    "question": "What is the primary difference between a `while` loop and a `for` loop, as described in the text?",
    "options": {
      "A": "A `while` loop iterates over a container object, while a `for` loop requires a counter variable.",
      "B": "A `while` loop continues as long as a condition is true, while a `for` loop iterates over elements in a container or a sequence.",
      "C": "A `while` loop is used for nested loops only, while a `for` loop is used for single loops.",
      "D": "A `while` loop can only count upwards, while a `for` loop can count both upwards and downwards."
    },
    "correct_option": "B"
  },
  {
    "question": "According to the text, what happens when a `break` statement is encountered inside a loop?",
    "options": {
      "A": "The loop restarts from the beginning.",
      "B": "The loop skips the current iteration and continues to the next.",
      "C": "The loop terminates immediately, and the program execution continues with the nex

## Scenario 2

In [60]:
scenario_2_prompt = ""
with open('scenario-2-prompts.txt', 'r') as file:
    scenario_2_prompt = file.read()

question_generator_config_1_shot = setup_question_generator_config(scenario_2_prompt)

In [61]:
contents_scenario_2 = [
    types.Content(
        role="user",
        parts=[
            types.Part.from_uri(
                file_uri=files[0].uri,
                mime_type=files[0].mime_type,
            ),
            types.Part.from_text(text="""Here is an example of the desired output format:
Example Output:
[
  {
    "question_text": "What is the output of the code snippet?\n`count = 0\nwhile count < 3:\n  print(count)\n  count += 1`",
    "options": {
      "A": "0 1 2 3",
      "B": "1 2 3",
      "C": "0 1 2",
      "D": "0 1"
    },
    "correct_option": "C"
  }
]

Now, using that exact JSON format, generate exactly 5 MCQs covering the different loop concepts and examples presented **in the provided file**.
"""),
        ]
    )
]

In [62]:
result_1_shot = client.models.generate_content(
  model=base_model,
  contents=contents_scenario_2,
  config=question_generator_config_1_shot,
)

In [63]:
print(f"{result_1_shot.text}")

[
  {
    "question_text": "What is the primary purpose of a `while` loop?",
    "options": {
      "A": "To execute a block of code a fixed number of times.",
      "B": "To iterate over elements in a container.",
      "C": "To execute a block of code repeatedly as long as a given condition is true.",
      "D": "To define a function."
    },
    "correct_option": "C"
  },
  {
    "question_text": "What happens when a `break` statement is encountered inside a loop?",
    "options": {
      "A": "The loop restarts from the beginning.",
      "B": "The current iteration is skipped, and the loop continues with the next iteration.",
      "C": "The loop terminates immediately.",
      "D": "The program exits entirely."
    },
    "correct_option": "C"
  },
  {
    "question_text": "What is a nested loop?",
    "options": {
      "A": "A loop that has no body.",
      "B": "A loop that contains one or more loops within its body.",
      "C": "A loop that only iterates over strings.",
    

## Scenario 3 & 4 Setup

In [64]:
content_with_initial_mcqs_1 = contents_scenario_1
content_with_initial_mcqs_2 = contents_scenario_2

content_with_initial_mcqs_1.append(
  types.Content(
    role="model",
    parts=[
      types.Part.from_text(text=f"""Evaluate the following 5 generated MCQs based on the criteria specified in your instructions (Topic Coverage/Relevance, Question Quality/Clarity, Answer Quality/Distractors, Correctness Verification - scale 1-5). 
Provide the scores, a brief comment for each question, and an overall topic coverage comment. 
Use ONLY the specified JSON output format. 
Generated MCQs: {result.text}
"""),
    ]
  )
)

content_with_initial_mcqs_2.append(
  types.Content(
    role="model",
    parts=[
      types.Part.from_text(text=f"""Here is an example of the desired JSON evaluation output format:
Example Evaluation Output:
{{
  'evaluation_results': [
    {{
      'question_evaluated': 'What does the `break` statement do inside a loop?',
      'evaluation': {{
        'topic_coverage_relevance_score': 5,
        'question_quality_clarity_score': 5,
        'answer_quality_distractors_score': 4,
        'correctness_verification_score': 5,
        'brief_comment': 'Tests fundamental loop control. Clear question. Distractors plausible.'
      }}
    }}
  ],
  'overall_topic_coverage_comment': 'The single example question covers loop control well, but a full set would need to cover loop types too.'
}}

Now, using that exact JSON format, evaluate the following 5 generated MCQs based on the criteria specified in your instructions (Topic Coverage/Relevance, Question Quality/Clarity, Answer Quality/Distractors, Correctness Verification - scale 1-5). Provide the scores, a brief comment for each question, and an overall topic coverage comment.

Generated MCQs: {result_1_shot.text}"""),
    ]
  )
)

## Scenario 3

In [65]:
evaluator_0_shot_prompt = ""
with open('evaluator-0-shot-prompt.txt', 'r') as file: 
    evaluator_0_shot_prompt = file.read()

content_with_initial_mcqs_1.append(
    types.Content(
        role="user",
        parts=[
            types.Part.from_text(text="Evaluate the following MCQs based on the context."),
        ]
    )
)

evaluator_agent_config_1 = types.GenerateContentConfig(
    response_mime_type="application/json",
    system_instruction=[
        types.Part.from_text(text=evaluator_0_shot_prompt),
    ],
)

In [66]:
evaluator_result_1 = client.models.generate_content(
  model=base_model,
  contents=content_with_initial_mcqs_1,
  config=evaluator_agent_config_1,
)

In [67]:
print(f">> evaluator_result_1: {evaluator_result_1.text}")

>> evaluator_result_1: {
  "evaluation_results": [
    {
      "question_id": 1,
      "topic_coverage": 5,
      "question_quality": 5,
      "answer_quality": 5,
      "correctness_verification": 5,
      "comments": "Covers the key difference between for and while loops accurately. Question and options are clear and well-defined. Distractors are plausible."
    },
    {
      "question_id": 2,
      "topic_coverage": 5,
      "question_quality": 5,
      "answer_quality": 5,
      "correctness_verification": 5,
      "comments": "Tests understanding of the break statement's behavior. Options are clear and represent common misconceptions."
    },
    {
      "question_id": 5,
      "topic_coverage": 5,
      "question_quality": 5,
      "answer_quality": 5,
      "correctness_verification": 5,
      "comments": "Tests understanding of the continue statement. Options are clearly distinct and plausible."
    },
    {
      "question_id": 4,
      "topic_coverage": 5,
      "question_qu

In [68]:
content_with_initial_mcqs_1.append(
  types.Content(
    role="model",
    parts=[
      types.Part.from_text(text=f"Feedback from evaluator: {evaluator_result_1.text}"),
    ]
  ),
)
content_with_initial_mcqs_1.append(
  types.Content(
    role="user",
    parts=[
      types.Part.from_text(text="Regenerate MCQs based on the feedback.")
    ]
  )
)

In [69]:
# Send feedback to question generator
result_with_feedback_1 = client.models.generate_content(
  model=base_model,
  contents=content_with_initial_mcqs_1,
  config=question_generator_config_0_shot,
)

In [70]:
print(f">> result_with_feedback_1: {result_with_feedback_1.text}")

>> result_with_feedback_1: [
  {
    "question": "According to the text, which loop type is best suited for iterating over a predetermined list of student names?",
    "options": {
      "A": "The `while` loop, because it can count upwards and downwards.",
      "B": "The `for` loop, because it is designed for iterating over container objects.",
      "C": "Both `for` and `while` loops are equally suitable for iterating over lists.",
      "D": "A nested `while` loop."
    },
    "correct_option": "B"
  },
  {
    "question": "Consider the following code: `counter = 1; while True: if counter >= 10: break; print(counter); counter += 1`. What is the purpose of the `break` statement in this code?",
    "options": {
      "A": "To skip printing the counter when its value is greater than or equal to 10.",
      "B": "To ensure the loop always executes at least once.",
      "C": "To prevent the loop from becoming an infinite loop by terminating it when the counter reaches 10.",
      "D": "

## Scenario 4

In [71]:
evaluator_1_shot_prompt = ""
with open('evaluator-1-shot-prompt.txt', 'r') as file: 
    evaluator_1_shot_prompt = file.read()

content_with_initial_mcqs_2.append(
    types.Content(
        role="user",
        parts=[
            types.Part.from_text(text="Evaluate the following MCQs based on the context."),
        ]
    )
)

evaluator_agent_config_2 = types.GenerateContentConfig(
    response_mime_type="application/json",
    system_instruction=[
        types.Part.from_text(text=evaluator_1_shot_prompt),
    ],
)

In [72]:
evaluator_result_2 = client.models.generate_content(
  model=base_model,
  contents=content_with_initial_mcqs_2,
  config=evaluator_agent_config_2,
)

In [73]:
print(f">> evaluator_result_2: {evaluator_result_2.text}")

>> evaluator_result_2: {
  "evaluation_results": [
    {
      "question_evaluated": "What is the primary purpose of a `while` loop?",
      "evaluation": {
        "topic_coverage_relevance_score": 5,
        "question_quality_clarity_score": 5,
        "answer_quality_distractors_score": 5,
        "correctness_verification_score": 5,
        "brief_comment": "Covers fundamental `while` loop purpose well. Options are clear and distinct; plausible distractors."
      }
    },
    {
      "question_evaluated": "What happens when a `break` statement is encountered inside a loop?",
      "evaluation": {
        "topic_coverage_relevance_score": 5,
        "question_quality_clarity_score": 5,
        "answer_quality_distractors_score": 5,
        "correctness_verification_score": 5,
        "brief_comment": "Addresses a key loop control feature. Clearly worded and correct answer. Good distractors."
      }
    },
    {
      "question_evaluated": "What is a nested loop?",
      "evaluatio

In [74]:
content_with_initial_mcqs_2.append(
  types.Content(
    role="model",
    parts=[
      types.Part.from_text(text=f"Feedback from evaluator: {evaluator_result_2.text}")
    ]
  ),
)

content_with_initial_mcqs_2.append(
  types.Content(
    role="user",
    parts=[
      types.Part.from_text(text="Regenerate MCQs based on the feedback.")
    ]
  )
)

In [75]:
print(">> Content with initial MCQs 2:")
for content in content_with_initial_mcqs_2:
    print(f"\nRole: {content.role}")
    for part in content.parts:
        print(f"Part text: {part.text}")
    print("-" * 50)

>> Content with initial MCQs 2:

Role: user
Part text: None
Part text: Here is an example of the desired output format:
Example Output:
[
  {
    "question_text": "What is the output of the code snippet?
`count = 0
while count < 3:
  print(count)
  count += 1`",
    "options": {
      "A": "0 1 2 3",
      "B": "1 2 3",
      "C": "0 1 2",
      "D": "0 1"
    },
    "correct_option": "C"
  }
]

Now, using that exact JSON format, generate exactly 5 MCQs covering the different loop concepts and examples presented **in the provided file**.

--------------------------------------------------

Role: model
Part text: Here is an example of the desired JSON evaluation output format:
Example Evaluation Output:
{
  'evaluation_results': [
    {
      'question_evaluated': 'What does the `break` statement do inside a loop?',
      'evaluation': {
        'topic_coverage_relevance_score': 5,
        'question_quality_clarity_score': 5,
        'answer_quality_distractors_score': 4,
        'corre

In [76]:
result_after_feedback_2 = client.models.generate_content(
  model=base_model,
  contents=content_with_initial_mcqs_2,
  config=question_generator_config_1_shot,
)

In [77]:
print(f">> result_after_feedback_2: {result_after_feedback_2.text}")

>> result_after_feedback_2: [
  {
    "question_text": "A `while` loop continues to execute its block of code as long as what condition is met?",
    "options": {
      "A": "The loop counter is greater than zero.",
      "B": "The loop expression evaluates to True.",
      "C": "The loop expression evaluates to False.",
      "D": "The `break` statement is not encountered."
    },
    "correct_option": "B"
  },
  {
    "question_text": "What is the immediate effect of a `continue` statement within a loop?",
    "options": {
      "A": "The loop terminates entirely.",
      "B": "The current iteration is skipped, and execution jumps to the next iteration's loop expression evaluation.",
      "C": "The program terminates.",
      "D": "The loop restarts from the beginning."
    },
    "correct_option": "B"
  },
  {
    "question_text": "In a nested loop, which loop controls the number of times the inner loop executes fully?",
    "options": {
      "A": "The inner loop.",
      "B": "Th